In [ ]:
import os
import sys
sys.path.append("../framework/")
sys.path.append("../sl/")
sys.path.append("../sid/")

import numpy as np
import matplotlib.pylab as plt
import pandas as pd

from sid_environment import SidEnvironment
from sid_environment_factory import SidEnvironmentFactory
from sid_build_parameter import SidBuildParameter
from sid_build_parameter_factory import SidBuildParameterFactory
from builder import Builder
from sid_evaluator import SidEvaluator
from store import Store
from sid_agent_factory import SidAgentFactory
from sid_trainer_factory import SidTrainerFactory
from mylogger import MyLogger

from loader import Loader

In [ ]:
trainingHistoryDbPath = "training_history.sqlite"

## S400: Load trained agents to analyze them

### SS410: initialize loader

In [ ]:
agentFactory = SidAgentFactory()
environmentFactory = SidEnvironmentFactory()
trainerFactory = SidTrainerFactory()

store = Store(trainingHistoryDbPath)

buildParameterFactory = SidBuildParameterFactory()
loader = Loader(agentFactory=agentFactory
                , environmentFactory=environmentFactory
                , trainerFactory=trainerFactory
                , buildParameterFactory=buildParameterFactory
                , store = store)

### SS420: initialize evaluator

In [ ]:
evaluator = SidEvaluator()

### SS420: check the distribution of sids

In [ ]:
tbl = {name: []  for name in SidEvaluator.names}
tbl["epoch"] = []
for agent, buildParameter, epoch, environment, trainer in loader.load("test%"):
    row = evaluator.evaluate(agent, buildParameter, epoch, environment, trainer)
    for (name, val) in zip(SidEvaluator.names, row):
        tbl[name].append(val)
    tbl["epoch"].append(epoch)
    for key in buildParameter.__dict__:
        if not key in tbl:
            tbl[key] = []
        tbl[key].append(buildParameter.__dict__[key])
tbl = pd.DataFrame(tbl)

In [ ]:
tbl